In [4]:
import time
import pandas as pd 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [8]:
url = 'https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=distance&myX=37.5605672&myY=126.9433486&searchKeyword='
driver = webdriver.Chrome()
driver.get(url)

In [9]:
driver.find_element(By.ID, 'searchKeyword').send_keys('서울')
driver.find_element(By.ID, 'fsSearchBtn').click()

In [10]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
lis = soup.select('.stations-list > li')
len(lis)

10

In [11]:
li = lis[0]

In [12]:
title = li.select_one('.title').get_text().strip()
title

'동작소방서'

In [13]:
addr = li.select_one('address').get_text().strip()
addr

'서울특별시 동작구 여의대방로16길 55(신대방동)'

In [14]:
tel = li.select_one('.tel').get_text().strip()
tel

'02-847-1190'

In [16]:
data = []
for li in lis:
    title = li.select_one('.title').get_text().strip()
    addr = li.select_one('address').get_text().strip()
    tel = li.select_one('.tel').get_text().strip()
    data.append({'기관명':title, '주소' :addr, '전화번호' : tel})
df = pd.DataFrame(data)

In [17]:
df

,기관명,주소,전화번호
0,동작소방서,서울특별시 동작구 여의대방로16길 55(신대방동),02-847-1190
1,서대문소방서,서울특별시 서대문구 연희로 182(연희동),02-3144-1190
2,광진소방서,서울특별시 광진구 광나루로 480(구의동),02-457-0119
3,송파소방서,서울특별시 송파구 오금로51길 56(마천동),02-403-2119
4,양천소방서,서울특별시 양천구 목동서로 180(목동),02-2655-1119
5,은평소방서,서울특별시 은평구 통일로 962(진관동),02-355-0119
6,노원소방서,서울특별시 노원구 한글비석로 1길 8(하계동),02-977-0119
7,종로소방서,서울특별시 종로구 종로1길 28(수송동),02-735-6119
8,도봉소방서,서울특별시 도봉구 도봉로 666(방학동),02-3492-3437
9,용산소방서,서울특별시 용산구 한강대로 167(한강로2가),02-794-0119


In [20]:
# //*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[5]/a
driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[5]/a').click()

In [22]:
driver.find_element(By.CSS_SELECTOR, '.next_page').click()

## 모든 페이지에 적용

In [ ]:
# 첫번째 페이지
driver.get(url)
time.sleep(2)
driver.find_element(By.ID, 'searchKeyword').send_keys('서울')
driver.find_element(By.ID, 'fsSearchBtn').click()

In [25]:
# 서울 100건
count = driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/div/p/strong[2]').text
count = int(count[:-1])
count

100

In [30]:
import math
total_pages = math.ceil(count/10)
total_pages

10

In [31]:
data1 = []
for i in range(total_pages):
    if i % 2 == 1:
        driver.find_element(By.CSS_SELECTOR, '.next_page').click()
        time.sleep(1)
    if i > 0 and i % 2 == 0:
        driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[5]/a').click()
        time.sleep(1)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select('.stations-list > li')
    for li in lis:
        title = li.select_one('.title').get_text().strip()
        addr = li.select_one('address').get_text().strip()
        tel = li.select_one('.tel').get_text().strip()
        data1.append({'기관명':title, '주소' :addr, '전화번호' : tel})

In [32]:
df1 = pd.DataFrame(data1)
df1.head()

,기관명,주소,전화번호
0,마포소방서,서울특별시 마포구 창전로 76(신수동),02-716-0604
1,강서소방서,서울특별시 강서구 양천로 550(등촌동),02-3663-0119
2,구로소방서,서울특별시 구로구 경인로 408(고척동),02-2618-0119
3,성동소방서,서울특별시 성동구 살곶이길 331(행당동),02-2622-1777
4,관악소방서,서울특별시 관악구 관악로 97(봉천동),02-874-8119


In [33]:
len(df1)

100

In [34]:
df1.to_csv('./data/서울소방서.csv', index=False)